In [2]:
!pip install scipy==1.13.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 458.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.6 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
Could not fetch URL https://pypi.ngc.nvidia.com/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.ngc.nvidia.com', port=443): Max retries exceeded with url: /pip/ (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)'))) - skipping


In [5]:
import gdown
import os
import zipfile


file_id = "19ZQV9Wf1CDKYJTv2QBLivPpojbygQ62c"
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, output="MRI2CT_training_npy_4_images.zip", quiet=False)

# Step 2: Extract the ZIP file
with zipfile.ZipFile("MRI2CT_training_npy_4_images.zip", 'r') as zip_ref:
    zip_ref.extractall("Evaluation/DAR-UNet-2-edition-4")

Downloading...
From (original): https://drive.google.com/uc?id=19ZQV9Wf1CDKYJTv2QBLivPpojbygQ62c
From (redirected): https://drive.google.com/uc?id=19ZQV9Wf1CDKYJTv2QBLivPpojbygQ62c&confirm=t&uuid=f0ebe580-98e2-411a-910e-0375aff62ed2
To: /home/jovyan/MRI2CT_training_npy_4_images.zip

  0%|                                                                                                                                                              | 0.00/528M [00:00<?, ?B/s]
  0%|▏                                                                                                                                                     | 524k/528M [00:00<08:27, 1.04MB/s]
  0%|▎                                                                                                                                                    | 1.05M/528M [00:00<04:29, 1.95MB/s]
  0%|▌                                                                                                                                         

In [6]:
file_id = "1PUdRL5v7JRVPa0ztUCWY03WkCh-Mzjp_"
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, output="CT_Images.zip", quiet=False)

# Step 2: Extract the ZIP file
with zipfile.ZipFile("CT_Images.zip", 'r') as zip_ref:
    zip_ref.extractall("Evaluation")

Downloading...
From (original): https://drive.google.com/uc?id=1PUdRL5v7JRVPa0ztUCWY03WkCh-Mzjp_
From (redirected): https://drive.google.com/uc?id=1PUdRL5v7JRVPa0ztUCWY03WkCh-Mzjp_&confirm=t&uuid=5ef99ae0-7bd5-4bc3-8824-9d582a840b1c
To: /home/jovyan/CT_Images.zip

  0%|                                                                                                                                                              | 0.00/724M [00:00<?, ?B/s]
  0%|                                                                                                                                                       | 524k/724M [00:00<13:31, 891kB/s]
  0%|▏                                                                                                                                                    | 1.05M/724M [00:00<06:59, 1.72MB/s]
  0%|▍                                                                                                                                                    | 2.10M/

In [2]:
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from scipy.linalg import sqrtm
from numpy import cov, trace, iscomplexobj


In [3]:
class CT3DDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load the 3D image
        image = np.load(self.image_paths[idx])
        image_slices = []

        # Apply the transform to each 2D slice
        for i in range(image.shape[2]):
            slice_img = image[:, :, i].astype(np.float32)  # Convert to float32
            slice_img = np.stack([slice_img] * 3, axis=-1)  # Convert grayscale to 3 channels
            slice_img = torch.tensor(slice_img).permute(2, 0, 1)  # Convert to torch.Tensor and rearrange dimensions
            if self.transform:
                slice_img = self.transform(slice_img)
            image_slices.append(slice_img)

        # Stack slices to form the final 3D tensor
        image_tensor = torch.stack(image_slices)
        return image_tensor

In [4]:
# Define the transform (normalize the image slices)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [5]:
# Load a pre-trained model (e.g., Inception v3)
model = models.inception_v3(pretrained=True, transform_input=False)
model.fc = nn.Identity()  # Remove the final classification layer
model.eval()

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [6]:
def extract_features(loader):
    features = []
    with torch.no_grad():
        for images in loader:
            # Flatten the 3D images into 2D (batch_size, num_slices, channels, H, W)
            batch_size, num_slices, _, H, W = images.shape
            images = images.view(batch_size * num_slices, 3, H, W)

            # Extract features using the pre-trained model
            output = model(images)
            output = output.view(batch_size, num_slices, -1)  # Shape: (batch_size, num_slices, feature_dim)
            output = output.mean(dim=1)  # Average features across slices
            features.append(output.cpu().numpy())

    features = np.vstack(features)
    return features

In [7]:
def calculate_fid(real_features, fake_features):
    # Calculate the mean and covariance of real and fake features
    mu1, sigma1 = real_features.mean(axis=0), cov(real_features, rowvar=False)
    mu2, sigma2 = fake_features.mean(axis=0), cov(fake_features, rowvar=False)
    
    eps = 1e-5  # You can adjust this value
    sigma1 += np.eye(sigma1.shape[0]) * eps
    sigma2 += np.eye(sigma2.shape[0]) * eps

    # Calculate the squared difference between the means
    ssdiff = np.sum((mu1 - mu2) ** 2.0)

    # Calculate the sqrt of the product of covariances
    covmean = sqrtm(sigma1.dot(sigma2))

    # Handle imaginary numbers from sqrtm
    if iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


# Compute FID between Real CT images and MRI2CT images - Author's model

In [7]:
# Paths to your images (replace with your actual paths)
real_image_paths = [os.path.join('Evaluation/CT_Images', f) for f in os.listdir('Evaluation/CT_Images')]
fake_image_paths = [os.path.join('Evaluation/DAR-UNet-main/MRI2CT_npy', f) for f in os.listdir('Evaluation/DAR-UNet-main/MRI2CT_npy')]

# Create datasets and dataloaders
real_dataset = CT3DDataset(real_image_paths, transform=transform)
fake_dataset = CT3DDataset(fake_image_paths, transform=transform)

real_loader = DataLoader(real_dataset, batch_size=1, shuffle=False)
fake_loader = DataLoader(fake_dataset, batch_size=1, shuffle=False)


In [8]:
# Extract features for real and fake images
real_features = extract_features(real_loader)
fake_features = extract_features(fake_loader)


In [9]:
# Calculate FID
fid_value = calculate_fid(real_features, fake_features)
print(f'FID: {fid_value}')

FID: 62.59908000242669


# Compute FID between Real CT images and MRI2CT images - aein's model - edition 4

In [21]:
import os
import zipfile

with zipfile.ZipFile("Evaluation/DAR-UNet-2-edition-4/MRI2CT_training_npy_5_images.zip", 'r') as zip_ref:
    zip_ref.extractall("Evaluation/DAR-UNet-2-edition-4/MRI2CT")

In [11]:
# Paths to your images (replace with your actual paths)
real_image_paths = [os.path.join('Evaluation/CT_Images', f) for f in os.listdir('Evaluation/CT_Images')]
fake_image_paths = [os.path.join('Evaluation/DAR-UNet-2-edition-4/MRI2CT_training_npy_4_images', f) for f in os.listdir('Evaluation/DAR-UNet-2-edition-4/MRI2CT_training_npy_4_images')]

# Create datasets and dataloaders
real_dataset = CT3DDataset(real_image_paths, transform=transform)
fake_dataset = CT3DDataset(fake_image_paths, transform=transform)

real_loader = DataLoader(real_dataset, batch_size=1, shuffle=False)
fake_loader = DataLoader(fake_dataset, batch_size=1, shuffle=False)

In [12]:
# Extract features for real and fake images
real_features = extract_features(real_loader)
fake_features = extract_features(fake_loader)

In [13]:
# Calculate FID
fid_value = calculate_fid(real_features, fake_features)
print(f'FID: {fid_value}')

FID: 49.4928377081978


# Compute FID between Real CT images and MRI2CT images - aein's model - edition 5

In [15]:
import gdown
import os
import zipfile

with zipfile.ZipFile("Evaluation/DAR-UNet-2-edition-5/MRI2CT_training_npy_edition_5.zip", 'r') as zip_ref:
    zip_ref.extractall("Evaluation/DAR-UNet-2-edition-5/MRI2CT")

In [16]:
# Paths to your images (replace with your actual paths)
real_image_paths = [os.path.join('Evaluation/CT_Images', f) for f in os.listdir('Evaluation/CT_Images')]
fake_image_paths = [os.path.join('Evaluation/DAR-UNet-2-edition-5/MRI2CT', f) for f in os.listdir('Evaluation/DAR-UNet-2-edition-5/MRI2CT')]

# Create datasets and dataloaders
real_dataset = CT3DDataset(real_image_paths, transform=transform)
fake_dataset = CT3DDataset(fake_image_paths, transform=transform)

real_loader = DataLoader(real_dataset, batch_size=1, shuffle=False)
fake_loader = DataLoader(fake_dataset, batch_size=1, shuffle=False)

In [17]:
# Extract features for real and fake images
real_features = extract_features(real_loader)
fake_features = extract_features(fake_loader)

In [18]:
# Calculate FID
fid_value = calculate_fid(real_features, fake_features)
print(f'FID: {fid_value}')

FID: 98.48893135308768


# Compute FID between Real CT images and MRI2CT images - aein's model - edition 8-v8-change SvS

In [20]:
# Paths to your images (replace with your actual paths)
real_image_paths = [os.path.join('Evaluation/CT_Images', f) for f in os.listdir('Evaluation/CT_Images')]
fake_image_paths = [os.path.join('output_i2i_v8_newSVS/MRI2CT_imgs_npy_ckpt42', f) for f in os.listdir('output_i2i_v8_newSVS/MRI2CT_imgs_npy_ckpt42')]

# Create datasets and dataloaders
real_dataset = CT3DDataset(real_image_paths, transform=transform)
fake_dataset = CT3DDataset(fake_image_paths, transform=transform)

real_loader = DataLoader(real_dataset, batch_size=1, shuffle=False)
fake_loader = DataLoader(fake_dataset, batch_size=1, shuffle=False)

In [21]:
# Extract features for real and fake images
real_features = extract_features(real_loader)
fake_features = extract_features(fake_loader)

In [22]:
# Calculate FID
fid_value = calculate_fid(real_features, fake_features)
print(f'FID: {fid_value}')

FID: 58.39149314766895


# Compute FID between Real CT images and MRI2CT images - aein's model - edition 8-v9-change SvS

In [8]:
# Paths to your images (replace with your actual paths)
real_image_paths = [os.path.join('Evaluation/CT_Images', f) for f in os.listdir('Evaluation/CT_Images')]
fake_image_paths = [os.path.join('output_i2i_v9_newSVS/MRI2CT_npy_ckpt36_imgs', f) for f in os.listdir('output_i2i_v9_newSVS/MRI2CT_npy_ckpt36_imgs')]

# Create datasets and dataloaders
real_dataset = CT3DDataset(real_image_paths, transform=transform)
fake_dataset = CT3DDataset(fake_image_paths, transform=transform)

real_loader = DataLoader(real_dataset, batch_size=1, shuffle=False)
fake_loader = DataLoader(fake_dataset, batch_size=1, shuffle=False)

In [9]:
# Extract features for real and fake images
real_features = extract_features(real_loader)
fake_features = extract_features(fake_loader)

In [10]:
# Calculate FID
fid_value = calculate_fid(real_features, fake_features)
print(f'FID: {fid_value}')

FID: 103.98062373202929


In [24]:
import shutil

# Define the folder to zip and the output zip file
folder_to_zip = "/home/jovyan/output_i2i_v8_newSVS/MRI2CT_npy_ckpt42"
output_filename = "MRI2CT_npy_i2i_v8_newSVS_ckpt42.zip"

# Create a zip archive
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)


'/home/jovyan/MRI2CT_npy_i2i_v8_newSVS_ckpt42.zip'

In [ ]:
http://jupiter.ferdowsi.cloud:2226/lab/tree/MRI2CT_npy_i2i_v8_newSVS_ckpt42.zip